# Chapter 25

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# install Pint if necessary

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# download modsim.py if necessary

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# import functions from modsim

from modsim import *

Intro

In the previous chapter we modeled a scenario with constant angular
velocity. In this chapter we make it more complex; we'll model a teapot, on a turntable, revolving with constant angular acceleration and deceleration.

## Angular acceleration

Just as linear acceleration is the derivative of velocity, **angular
acceleration** is the derivative of angular velocity. And just as linear acceleration is caused by force, angular acceleration is caused by the rotational version of force, **torque**. If you are not familiar with torque, you can read about it at <http://modsimpy.com/torque>.

In general, torque is a vector quantity, defined as the **cross
product** of $\vec{r}$ and $\vec{F}$, where $\vec{r}$ is the **lever
arm**, a vector from the point of rotation to the point where the force is applied, and $\vec{F}$ is the vector that represents the magnitude and direction of the force.

However, for the problems in this chapter, we only need the *magnitude* of torque; we don't care about the direction. In that case, we can compute 

$$\tau = r F \sin \theta$$ 

where $\tau$ is torque, $r$ is the length of the lever arm, $F$ is the magnitude of force, and $\theta$ is the angle between $\vec{r}$ and $\vec{F}$.

Since torque is the product of a length and a force, it is expressed in newton meters (Nm).

## Moment of inertia

In the same way that linear acceleration is related to force by Newton's second law of motion, $F=ma$, angular acceleration is related to torque by another form of Newton's law: 

$$\tau = I \alpha$$ 

where $\alpha$ is angular acceleration and $I$ is **moment of inertia**. Just as mass is what makes it hard to accelerate an object, moment of inertia is what makes it hard to spin an object.
(That might sound like a dumb way to describe mass, but might actually be the most fundamental definition.)

In the most general case, a 3-D object rotating around an arbitrary
axis, moment of inertia is a tensor, which is a function that takes a
vector as a parameter and returns a vector as a result.

Fortunately, in a system where all rotation and torque happens around a single axis, we don't have to deal with the most general case. We can treat moment of inertia as a scalar quantity.

For a small object with mass $m$, rotating around a point at distance
$r$, the moment of inertia is $I = m r^2$. For more complex objects, we can compute $I$ by dividing the object into small masses, computing
moments of inertia for each mass, and adding them up.

However, for most simple shapes, people have already done the
calculations; you can just look up the answers. For example, see
<http://modsimpy.com/moment>.

## Teapots and turntables

Tables in Chinese restaurants often have a rotating tray or turntable
that makes it easy for customers to share dishes. These turntables are
supported by low-friction bearings that allow them to turn easily and
glide. However, they can be heavy, especially when they are loaded with food, so they have a high moment of inertia.

Suppose I am sitting at a table with a pot of tea on the turntable
directly in front of me, and the person sitting directly opposite asks
me to pass the tea. I push on the edge of the turntable with 1 N of
force until it has turned 0.5 rad, then let go. The turntable glides
until it comes to a stop 1.5 rad from the starting position. How much
force should I apply for a second push so the teapot glides to a stop
directly opposite me?

We'll answer this question in these steps:

1.  I'll use the results from the first push to estimate the coefficient of friction for the turntable.

2.  As an exercise, you'll use that coefficient of friction to estimate the force needed to rotate the turntable through the remaining angle.

Our simulation will use the following parameters:

1.  The radius of the turntable is 0.5 m, and its weight is 7 kg.

2.  The teapot weights 0.3 kg, and it sits 0.4 m from the center of the turntable.

![Diagram of a turntable with a
teapot.](figs/teapot.pdf){height="2.5in"}

This figure shows the scenario, where $F$ is the force I apply to the turntable at the perimeter, perpendicular to the moment arm, $r$, and $\tau$ is the resulting torque. The blue circle near the bottom is the teapot.

Here's a `Params` object with the parameters of the scenario:

In [2]:
import pint

units = pint.UnitRegistry()
rad = units.radian
m = units.meter
s = units.second
kg = units.kilogram
N = units.newton

In [3]:
from modsim import Params
from numpy import pi

params = Params(radius_disk=0.5,     # m
                mass_disk=7,         # kg
                radius_pot=0.4,      # m
                mass_pot=0.3,        # kg
                force=1,             # N
                torque_friction=0.2, # N*m
                theta_0=0,           # radian
                theta_push=0.5,      # radian
                theta_test=1.5,      # radian
                theta_target=pi,     # radian
                t_end=20             # s
               )

In [4]:
from modsim import Params
from numpy import pi

params = Params(radius_disk = 0.5*m,
                mass_disk = 7*kg,
                radius_pot = 0.4*m,
                mass_pot = 0.3*kg,
                force = 1*N,
                torque_friction = 0.2*N*m,
                theta_0 = 0*rad,
                omega_0 = 0*rad/s,
                theta_push = 0.5*rad,
                theta_test = 1.5*rad,
                theta_target = pi*rad,
                t_end = 20*s)

`make_system` creates the initial state, `init`, and computes the total
moment of inertia for the turntable and the teapot.

In [5]:
from modsim import State, System

def make_system(params):
    """Make a system object.
    
    params: Params object
    
    returns: System object
    """
    mass_disk, mass_pot = params.mass_disk, params.mass_pot
    radius_disk, radius_pot = params.radius_disk, params.radius_pot
    
    init = State(theta=params.theta_0, omega=params.omega_0)
    
    I_disk = mass_disk * radius_disk**2 / 2
    I_pot = mass_pot * radius_pot**2
    
    return System(params, 
                  init=init, 
                  I=I_disk+I_pot,
                  )

In the initial state, `theta` represents the angle of the table in rad;
`omega` represents the angular velocity in rad/s.

`I_disk` is the moment of inertia of the turntable, which is based on
the moment of inertia for a horizontal disk revolving around a vertical axis through its center: 

$$I_{disk} = m r^2 / 2$$ 

`I_pot` is the moment
of inertia of the teapot, which I treat as a point mass with:

$$I_{point} = m r^2$$ 

In SI units, moment of inertia is expressed in kg m$^2$.

Now we can make a `System` object:

In [6]:
system1 = make_system(params)

Here's a slope that takes the current state, which contains angle and
angular velocity, and returns the derivatives, angular velocity and
angular acceleration:

In [7]:
def slope_func(t, state, system):
    """Computes the derivatives of the state variables.
    
    state: State object
    t: time
    system: System object 
    
    returns: sequence of derivatives
    """
    theta, omega = state
    radius_disk, force = system.radius_disk, system.force
    torque_friction, I = system.torque_friction, system.I
    
    torque = radius_disk * force - torque_friction
    alpha = torque / I
    
    return omega, alpha 

In this scenario, the force I apply to the turntable is always
perpendicular to the lever arm, so $\sin \theta = 1$ and the torque due
to force is $\tau = r F$.

`torque_friction` represents the torque due to friction. Because the
turntable is rotating in the direction of positive `theta`, friction
acts in the direction of negative `theta`.

In [8]:
slope_func(0, system1.init, system1)

We are almost ready to run the simulation, but first there's a problem we have to address.

When I stop pushing on the turntable, the angular acceleration changes
abruptly. We could implement the slope function with an `if` statement
that checks the value of `theta` and sets `force` accordingly. And for a coarse model like this one, that might be fine. But we will get more
accurate results if we simulate the system in two phases:

1.  During the first phase, force is constant, and we run until `theta` is 0.5 radians.

2.  During the second phase, force is 0, and we run until `omega` is 0.

Then we can combine the results of the two phases into a single
`TimeFrame`.

Here's the event function I'll use for Phase 1; it stops the simulation when `theta` reaches `theta_end`, which is when I stop pushing:

In [9]:
def event_func1(t, state, system):
    """Stops when theta reaches theta_push.
    
    state: State object
    t: time
    system: System object 
    
    returns: difference from target
    """
    theta, omega = state
    return theta - system.theta_push

As usual, we'll test the event function with the initial conditions.

In [10]:
event_func1(0, system1.init, system1)

Now we can run the first phase of the simulation.

In [11]:
from modsim import run_solve_ivp

results1, details1 = run_solve_ivp(system1, slope_func,
                                   events=event_func1)
details1.message

In [12]:
results1.tail()



Before we run the second phase, we have to extract the final time and
state of the first phase.

In [13]:
t_0 = results1.index[-1]
init2 = results1.iloc[-1]

Now we can make a `System` object for Phase 2, with the initial state
from Phase 1, and with `force=0`.

In [14]:
system2 = System(system1, t_0=t_0, init=init2, force=0)

For the second phase, we need an event function that stops when the
turntable stops; that is, when angular velocity is 0.

In [15]:
def event_func2(t, state, system):
    """Stops when omega is 0.
    
    state: State object
    t: time
    system: System object 
    
    returns: omega
    """
    theta, omega = state
    return omega

In [16]:
event_func2(system2.t_0, system2.init, system2)

Now we can run the second phase.

In [17]:
results2, details2 = run_solve_ivp(system2, slope_func,
                                   events=event_func2)
details2.message

Pandas provides `combine_first`, which combines `results1` and
`results2`.

In [18]:
results = results1.combine_first(results2)

Now we can plot `theta` for both phases.

In [19]:
from modsim import decorate

def plot_theta(results):
    results.theta.plot(label='theta')
    decorate(xlabel='Time (s)',
             ylabel='Angle (rad)')
    
plot_theta(results)

And `omega`.

In [20]:
def plot_omega(results):
    results.omega.plot(label='omega', color='C1')
    decorate(xlabel='Time (s)',
             ylabel='Angular velocity (rad/s)')
    
plot_omega(results)

Angular velocity, `omega`, increases linearly while I am pushing, and decreases linearly after I let go. The angle, `theta`, is the integral of angular velocity, so it forms a parabola during each phase.

In the next section, we'll use this simulation to estimate the torque
due to friction.

## Estimating friction

Let's take the code from the previous section and wrap it in a function.

In [21]:
def run_two_phases(force, torque_friction, params):
    """Run both phases.
    
    force: force applied to the turntable
    torque_friction: friction due to torque
    params: Params object
    
    returns: TimeFrame of simulation results
    """
    # put the specified parameters into the Params object
    params = Params(params, 
                    force=force, 
                    torque_friction=torque_friction)

    # run phase 1
    system1 = make_system(params)
    results1, details1 = run_solve_ivp(system1, slope_func, 
                                       events=event_func1)

    # get the final state from phase 1
    t_0 = results1.index[-1]
    init2 = results1.iloc[-1]
    
    # run phase 2
    system2 = System(system1, t_0=t_0, init=init2, force=0)
    results2, details2 = run_solve_ivp(system2, slope_func, 
                                       events=event_func2)
    
    # combine and return the results
    results = results1.combine_first(results2)
    return results

Let's test it with the same parameters.

In [22]:
force = 1
torque_friction = 0.2
results = run_two_phases(params.force, params.torque_friction, params)
results.tail()

And check the results.

In [23]:
theta_final = results.iloc[-1].theta
theta_final

We can use `run_two_phases` to write an error function we can use, with
`root_bisect`, to find the torque due to friction that yields the
observed results from the first push, a total rotation of 1.5 rad.

In [24]:
def error_func1(torque_friction, params):
    """Error function for root_scalar.
    
    torque_friction: hypothetical value
    params: Params object
    
    returns: offset from target value
    """
    force = 1
    results = run_two_phases(force, torque_friction, params)
    theta_final = results.iloc[-1].theta
    print(torque_friction, theta_final)
    return theta_final - params.theta_test

Testing the error function.

In [25]:
guess1 = 0.1
error_func1(guess1, params)

In [26]:
guess2 = 0.2
error_func1(guess2, params)

And running `root_scalar`.

In [27]:
from scipy.optimize import root_scalar

res = root_scalar(error_func1, params, bracket=[guess1, guess2])

The result is 0.166 Nm, a little less than the initial guess.

## Animation


Here's a draw function we can use to animate the results.

In [28]:
from matplotlib.patches import Circle, Arrow
from matplotlib.pyplot import gca, axis
from modsim import pol2cart

def draw_func(t, state, params):
    theta, omega = state
    
    # draw a circle for the table
    circle1 = Circle([0, 0], params.radius_disk)
    gca().add_patch(circle1)
    
    # draw a circle for the teapot
    center = pol2cart(theta, params.radius_pot)
    circle2 = Circle(center, 0.05, color='C1')
    gca().add_patch(circle2)

    axis('equal')

In [29]:
from modsim import remove_units

params_no_unit = remove_units(params)

In [30]:
state = results.iloc[0]
draw_func(0, state, params_no_unit)

In [31]:
from modsim import animate

animate(results, draw_func, params_no_unit)

## Summary

Now that we know the torque due to friction, we can compute the force
needed to rotate the turntable through the remaining angle, that is,
from 1.5 rad to 3.14 rad.






### Exercises

Now finish off the example by estimating the force that delivers the teapot to the desired position.

Write an error function that takes `force` and `params` and returns the offset from the desired angle.

In [32]:
# Solution goes here

In [33]:
# Solution goes here

In [34]:
# Solution goes here

In [35]:
# Solution goes here

In [36]:
# Solution goes here